In [1]:
import requests
import json
import pandas as pd
import time
import math
import itertools
from tqdm import tqdm

In [2]:
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
}

In [3]:
entries = []

In [4]:
size = 1024
x_start = int(math.floor(2738000 / size))
y_start = int(math.floor(1251000 / size))
x_end = int(math.ceil(2750000 / size))
y_end = int(math.ceil(1258000 / size))

In [5]:
print(f'making >= {((x_end - x_start + 1)) * ((y_end - y_start + 1))} requests')

making >= 126 requests


In [6]:
request_counter = {2**i: 0 for i in range(11)}

In [7]:
failed_tasks = []

In [8]:
def scrape_region(sx, sy, s):
    time.sleep(1)
    request_counter[s] += 1
    try:
        url = f"https://api3.geo.admin.ch/rest/services/all/MapServer/identify?geometry={sx+s//2}%2C{sy+s//2}&geometryFormat=geojson&geometryType=esriGeometryPoint&imageDisplay=1920%2C333%2C96&lang=de&layers=all%3Ach.bfs.gebaeude_wohnungs_register&limit=202&mapExtent={sx}%2C{sy}%2C{sx+s}%2C{sy+s}&returnGeometry=true&sr=2056&tolerance=100"
        r = requests.get(url=url, headers=HEADERS)
        result = json.loads(r.content)['results']
    except Exception as e:
        print(e)
        failed_tasks.append((sx, sy, s, e))
        return
    if len(result) <= 200 or s == 1:
        entries.extend(result)
    else:
        scrape_region(sx, sy, s//2)
        scrape_region(sx+s//2, sy, s//2)
        scrape_region(sx, sy+s//2, s//2)
        scrape_region(sx+s//2, sy+s//2, s//2)

In [9]:
for x, y in tqdm(list(itertools.product(range(x_start, x_end+1), range(y_start, y_end+1)))):
    scrape_region(x*size, y*size, size)

 47%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                                            | 59/126 [42:44<48:31, 43.46s/it]


KeyboardInterrupt: 

In [ ]:
request_counter

In [ ]:
print(f'made a total of {sum(request_counter.values())} requests')